In [1]:
import requests
from datetime import datetime
import urllib.parse
import pandas as pd

In [2]:
def fetch_wikidata(params):
    wikidata_url = "https://www.wikidata.org/w/api.php"
    try:
        response = requests.get(wikidata_url, params=params)
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"There was an error: {e}"

In [3]:
# Function to resolve Wikipedia redirects
def resolve_redirect(title):
    wikipedia_api_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "titles": title,
        "redirects": 1,
        "format": "json"
    }
    response = requests.get(wikipedia_api_url, params=params)
    data = response.json()

    # Check if a redirect exists and get the final title
    if 'redirects' in data['query']:
        final_title = data['query']['redirects'][0]['to']
    else:
        final_title = title  # No redirect, use the original title

    return final_title

In [4]:
# Function to get the Wikidata ID from a Wikipedia page title
def get_wiki_id_from_page(page_title):
    final_title = resolve_redirect(page_title)  # Resolve redirects first
    params = {
        "action": "wbgetentities",
        "format": "json",
        "sites": "enwiki",
        "titles": final_title,
        "languages": "en",
        "redirects": "yes",
    }
    data = fetch_wikidata(params)
    if isinstance(data, str) or 'entities' not in data or len(data['entities']) == 0:
        return None

    entity_id = list(data['entities'].keys())[0]
    return entity_id

In [5]:
# Load the CSV file
df = pd.read_csv('nndb.csv')

In [6]:
# Filter rows where WIKI_ID is -1
df_needs_update = df[df['WIKI_ID'] == '-1']

In [7]:
len(df_needs_update)

4912

In [8]:
# Apply the function only to rows that need update
df_needs_update['WIKI_ID'] = df_needs_update['WIKI_PAGE'].apply(get_wiki_id_from_page)

/var/folders/bq/7m4s9ggj7d1_d2gmbh7ffmf40000gn/T/ipykernel_64381/366946256.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_needs_update['WIKI_ID'] = df_needs_update['WIKI_PAGE'].apply(get_wiki_id_from_page)


In [9]:
# Merge the updated data back into the original dataframe
df.update(df_needs_update)

In [10]:
len(df)

36423

In [11]:
# Save the updated dataframe to a new CSV file
df.to_csv('updated_nndb_wiki_id.csv', index=False)